<a href="https://colab.research.google.com/github/usamaeltmsah/HR-Bot-evaluation-model/blob/universal-sentence-encoder/HR_Bot_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%%capture
# Install the latest Tensorflow version.sentence
!pip3 install --upgrade tensorflow-gpu
# Install TF-Hub.
!pip3 install tensorflow-hub

In [5]:
import tensorflow as tf
import tensorflow_hub as hub
import math
import numpy as np

# load the universal sentence encoder model
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)

module https://tfhub.dev/google/universal-sentence-encoder-large/5 loaded


In [6]:
class Evaulator:
  def __embed(self, input):
    return model(input)

  def __calculate_correlation(self, features):
    corrList = np.inner(features[:-1], features[-1])
    return corrList

  def __get_best_score(self, corrList):
    return np.max(corrList)

  def evalute_applicant_answer(self, model_answers, applicant_answer):
    answers = model_answers
    answers.append(applicant_answer)
    embedding_answers = self.__embed(answers)
    corr = self.__calculate_correlation(embedding_answers)
    score = self.__get_best_score(corr)
    """Returns the similarity scores"""
    return round(score*10,2)

In [ ]:
!pip install flask-ngrok
!pip install Wikipedia-API

In [ ]:
import wikipediaapi
from googlesearch import search
# from flask import render_template

class Parser:

    def __init__(self, skill):
        self.skill = skill

    def parse(self, sign):
        
        sentence = self.skill.split()
        search_key = ''
        for i in range(len(sentence)-1):
            search_key += sentence[i]
            search_key += sign
        # print(sentence[-1])    
        search_key += sentence[-1]

        return search_key

class FeedBack:
    
    def __init__(self, skills):
        self.skills = skills

        

    #Done
    def wikipediaSearch(self, skill):
        wiki_wiki = wikipediaapi.Wikipedia('en')
        temp = wiki_wiki.page(skill)
        summary = temp.summary
        
        return summary
    
    #Done
    def googleSearch(self):
        skills = self.skills
        question = self.question
        result = []

        for skill in skills:
            query = question + ' + ' + skill['name']
            for j in search(query, tld="com", num=10, stop=1, pause=2): 
                result.append(str(j)) 

        return result

    #Done
    def courseraSearch(self, skill):
        p = Parser(skill)
        search_key = p.parse('%20')
        link = 'https://www.coursera.org/search?query=' + search_key
        # links = []
        # result = []
        # for search_key in search_keys:
        #     link = 'https://www.coursera.org/search?query=' + search_key
        #     links.append(link)
        
        # for link in links :
        #     result.append(link)

        return link

    #Done
    def udemySearch(self, skill):
        p = Parser(skill)
        search_key = p.parse('+')
        link = 'https://www.udemy.com/courses/search/?q=' + search_key
        # links = []
        # result = []
        # for search_key in search_keys:
        #     link = 'https://www.udemy.com/courses/search/?q=' + search_key
        #     links.append(link)
        
        # for link in links :
        #     result.append(link)

        return link
    
    #Done
    def pluralsightSearch(self, skill):
        p = Parser(skill)
        search_key = p.parse('%20')
        link = 'https://www.pluralsight.com/search?q=' + search_key
        # links = []
        # result = []
        # for search_key in search_keys:
        #     link = 'https://www.pluralsight.com/search?q=' + search_key
        #     links.append(link)
        
        # for link in links :
        #     result.append(link)

        return link

    #Done
    def lyndaSearch(self, skill):
        p = Parser(skill)
        search_key = p.parse('%20')
        link = 'https://www.lynda.com/search?q=' + search_key
        # links = []
        # result = []
        # for search_key in search_keys:
        #     link = 'https://www.lynda.com/search?q=' + search_key
        #     links.append(link)
        
        # for link in links :
        #     result.append(link)

        return link

    def create_a_skill_summary(self, skill):
        summary = self.wikipediaSearch(skill)
        return summary
    
    def create_a_skill_links(self, skill):
        links = []
        links.append(self.courseraSearch(skill))
        links.append(self.udemySearch(skill))
        links.append(self.pluralsightSearch(skill))
        links.append(self.lyndaSearch(skill))

        
        # result = {
        #     "skill1":
        #     {
        #     "summary": "From Weki",
        #     "links": [
        #         "https://...", "https://...", "https://...", "https://..."
        #     ]
        # }

        # }
             
        return links
    def create_feedback(self):

        dictionary = {

        }
        for skill in self.skills:
            dictionary[skill] = {}
            dictionary[skill]["summary"] = self.create_a_skill_summary(skill)
            dictionary[skill]["links"] = self.create_a_skill_links(skill)
        return dictionary

# p = Parser('machine learning andrew eg')
# print(    p.parseForCoursera())
if __name__ == "__main__":
    # p = Parser("data base")
    # print(p.parse('+'))
    f = FeedBack(["data base", "machine learning"])
    print(f.create_feedback())

{'data base': {'summary': 'A database is an organized collection of data, generally stored and accessed electronically from a computer system. Where databases are more complex they are often developed using formal design and modeling techniques.\nThe database management system (DBMS) is the software that interacts with end users, applications, and the database itself to capture and analyze the data. The DBMS software additionally encompasses the core facilities provided to administer the database. The sum total of the database, the DBMS and the associated applications can be referred to as a "database system". Often the term "database" is also used to loosely refer to any of the DBMS, the database system or an application associated with the database.\nComputer scientists may classify database-management systems according to the database models that they support. Relational databases became dominant in the 1980s. These model data as rows and columns in a series of tables, and the vast 

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, abort, jsonify

evaluator = Evaulator()
app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run

@app.route('/')
def index():
    return "Machine Learning Models For HR-Bot"

@app.route('/evaluate', methods=['POST'])
def evaluation():
    if not request.json or not 'token' in request.json or not 'answers' in request.json or not 'applicant_answer' in request.json:
        abort(400)

    token = str(request.json["token"])

    if not(token == "hr_bot_2019_2020"):
        abort(404)
    else:
        model_answers = request.json['answers']
        applicant_answer = request.json['applicant_answer']
        score = evaluator.evalute_applicant_answer(model_answers, applicant_answer)
        print()
        return jsonify({'score': str(score)}), 200

# intput example : 
# {
#     "data" :
#     {
#         "question": "what is the desicion tree", 
#         "skills": 
#         [
#             {
#                 "name": "machine learning",
#                 "site": "www.datacamp.com"
#             },
#             {
#                 "name": "object oriented",
#                 "site":"www.datacamp.com"
#             }
#         ],
#         "token": "hr_bot_2019_2020"
#     }
# }
@app.route('/interview/feedback', methods=['POST'])
def feedback():
    if not request.json or not 'skills' in request.json or not 'token' in request.json :
        abort(400)

    token = str(request.json["token"])
    if not (token == "hr_bot_2019_2020"):
        abort(404)
    else:
        skills = (request.json['skills'])
        f = FeedBack(skills=skills)
        my_feedback = f.create_feedback()

        return jsonify({'feedback': my_feedback}), 200

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://9bee817def9f.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [12/Aug/2020 08:52:33] "POST /evaluate HTTP/1.1" 200 -


INFO:werkzeug:127.0.0.1 - - [12/Aug/2020 08:52:40] "POST /interview/feedback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Aug/2020 08:55:37] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Aug/2020 08:55:56] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Aug/2020 08:55:57] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [12/Aug/2020 09:01:39] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Aug/2020 09:01:39] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [12/Aug/2020 09:02:49] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Aug/2020 09:02:51] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Aug/2020 09:05:59] "POST /interview/feedback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Aug/2020 09:07:02] "POST /interview/feedback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Aug/2020 09:07:28] "GET /evaluate HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [12/Aug/2020 09:10:32] "POST /evaluate HTTP/1.1" 200 -


INFO:werkzeug:127.0.0.1 - - [12/Aug/2020 09:59:04] "POST /evaluate HTTP/1.1" 200 -


INFO:werkzeug:127.0.0.1 - - [12/Aug/2020 10:02:13] "POST /interview/feedback HTTP/1.1" 200 -
